In [ ]:
%reload_ext autoreload
%autoreload 2

import os
import re
import cv2
import tqdm
import numpy as np
import pandas as pd

import argus
from argus import load_model, Model

from src.utils import rle_decode, rle_encode

import torch
from torch.utils.data import Dataset, DataLoader

from skimage.feature import peak_local_max
from skimage.morphology import watershed, label
from scipy import ndimage

from src.models.unet_flex import UNetFlexProb
from src.losses import ShipLoss
from src.metrics import ShipIOUT
from src.utils import  filename_without_ext
from src.transforms import ProbOutputTransform, ImageToTensor
from src.dataset import ShipDataset


import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
save_path = '../../data/models/linknet34_folds_007/fold_0/'

BATCH_SIZE = 8
IMG_SIZE = (768, 768)

SEGM_THRESH = 0.5
DT_THRESH = 0.001
PROB_THRESH = 0.001

In [ ]:
def show_img(img):
    plt.figure(dpi=200)
    plt.imshow(img)
    plt.show()

def show_img_tensor(tensor):
    img = np.moveaxis(tensor.numpy(), 0, -1)[:,:,::-1]
    show_img(img)


def show_in_cols(masks_list, n_col=3):
    n_masks = len(masks_list)
    n_row = n_masks//n_col
    if n_masks % n_col > 0:
        n_row += 1
    
    f, ax = plt.subplots(n_row, n_col, figsize=(18,6))
    for i in range(n_masks):
        a = ax[i//n_col][i%n_col]
        a.imshow(masks_list[i])
        a.axis('off')


def show_trg_tensor(tensor):
    masks = tensor.numpy()
    masks_list = [masks[i, :, :] for i in range(masks.shape[0])]
    show_in_cols(masks_list)

In [ ]:
class ShipMetaModel(Model):
    nn_module = {
        'UNetFlexProb': UNetFlexProb,
    }
    loss = {
        'ShipLoss': ShipLoss
    }
    prediction_transform = {
        'ProbOutputTransform': ProbOutputTransform
    }

In [ ]:
def get_best_model_path(dir_path):
    model_scores = []
    for model_name in os.listdir(dir_path):
        score = re.search(r'-(\d+(?:\.\d+)?).pth', model_name)
        if score is not None:
            score = score.group(0)[1:-4]
            model_scores.append((model_name, score))
    model_score = sorted(model_scores, key=lambda x: x[1])
    best_model_name = model_score[-1][0]
    best_model_path = os.path.join(dir_path, best_model_name)
    return best_model_path

In [ ]:
# model = load_model(get_best_model_path(save_path))
model = load_model('/workdir/data/model-049-0.864456.pth')
model.nn_module.eval()

In [ ]:
test_dir = '../data/datasets/ships_small/test_small/'
test_ids = [filename_without_ext(file) for file in os.listdir(test_dir)]
print("Images for testing:", len(test_ids))
deploy_dataset = ShipDataset(test_ids, imgs_dir=test_dir, masks=False, image_transform=ImageToTensor())
deploy_loader = DataLoader(deploy_dataset, batch_size=BATCH_SIZE,
                           shuffle=False, num_workers=1)

In [ ]:
edge_min = 8
ellipticity_min = 5
eps = 1e-9
ell_f = 3
area_diff_thres = 100
def bin2uint(img):
    return np.uint8(img*255)

# https://stackoverflow.com/questions/42798659/how-to-remove-small-connected-objects-using-opencv
def clean_small(mask, thres=10):
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(bin2uint(mask), connectivity=4)
    sizes = stats[1:, -1];
    nb_components = nb_components - 1
    img2 = np.zeros((output.shape), dtype=np.bool)

    for i in range(nb_components):
        if sizes[i] >= thres:
            img2[output == i + 1] = True
    return img2

def max_area(mask):
    nb_components, output, stats, centroids = cv2.connectedComponentsWithStats(bin2uint(mask), connectivity=4)
    sizes = stats[1:, -1];
    return np.max(sizes)

def fit_rect(mask):
    j = 0
    all_masks = np.zeros_like(mask)
    for k in np.unique(mask)[1:]:
        _, contours, hierarchy = cv2.findContours(bin2uint(mask==k), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        for i in range(len(contours)):
            if hierarchy[0,i,3] == -1:
                epsilon = 0.1*cv2.arcLength(contours[i],True)
                approx = cv2.approxPolyDP(contours[i],epsilon,True)
                rect = cv2.minAreaRect(approx)
                if rect[1][0]<rect[1][1]:
                    hw = (rect[1][0]*F_H, rect[1][1]*F_W)
                else:
                    hw = (rect[1][0]*F_W, rect[1][1]*F_H)
                rect = (rect[0], hw, rect[2])
                box = cv2.boxPoints(rect)
                box = np.int0(box)
                filled_rectangle = cv2.fillPoly(np.zeros_like(mask), pts =[box], color=(1,1,1))
                all_masks[filled_rectangle > 0] = j+1
                j+=1

    return all_masks

def fill_con(markers):
    ret = np.zeros_like(markers)
    for i in np.unique(markers)[1:]:
        img = np.zeros((markers.shape), dtype=np.bool)
        img[markers == i] = True
        _, contours, hierarchy = cv2.findContours(bin2uint(img),
                                cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        img2 = np.zeros((markers.shape), dtype=np.uint8)
        for cnt in contours:
            hull = cv2.convexHull(cnt)
            img2 = cv2.fillConvexPoly(img2.copy(), hull, (255,255,255), 8)
        ret[img2>0] = i
    return ret

def dist_eq(mask):
    distance = np.zeros_like(mask, dtype=np.float32)
    _, markers = cv2.connectedComponents(mask)
    for i in np.unique(markers)[1:]:
        img = np.zeros((markers.shape), dtype=np.bool)
        img[markers == i] = True
        _, contours, hierarchy = cv2.findContours(bin2uint(img),
                                cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        img2 = np.zeros((markers.shape), dtype=np.uint8)
        for cnt in contours:
            hull = cv2.convexHull(cnt)
            img2 = cv2.fillConvexPoly(img2.copy(), hull, (255,255,255), 8)
        dt = cv2.distanceTransform(img2, distanceType=cv2.DIST_L2, maskSize=5)
        cv2.normalize(dt, dt, 0, 1.0, cv2.NORM_MINMAX)
        distance += dt
    return distance

kernel_shape = (3, 3)
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, kernel_shape)

def waters(mask, base, dt, boundaries):
    mask = clean_small(mask)
    mask_uint8 = bin2uint(mask)
    # Remove small FP objects on image edges
    labeled_array, num_features = ndimage.label(mask_uint8)
    objs = ndimage.find_objects(labeled_array)
    for obj in objs:
        for x, s in zip(obj, mask.shape):
            d = abs(x.stop - x.start)
            if (x.stop == s or x.start == 0) and d <= edge_min:
                mask[obj] = False
    mask_uint8 = bin2uint(mask)

    # Boundaries separation
    boundaries = clean_small(boundaries, 8)
    _, contours, hierarchy = cv2.findContours(bin2uint(boundaries),
                                              cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        if len(cnt) >4:
            ellipse = cv2.fitEllipse(cnt)
            if ellipse[1][0]/(ellipse[1][1]+eps) > ellipticity_min or ellipse[1][1]/(ellipse[1][0]+eps) > ellipticity_min:
                if ellipse[1][0] > ellipse[1][1]:
                    hw = (ellipse[1][0]*ell_f, 1)
                else:
                    hw = (1, ellipse[1][1]*ell_f)
                ellipse = (ellipse[0], hw, ellipse[2])
                box = cv2.boxPoints(ellipse)
                box = np.int0(box)
                mask_uint8 = cv2.drawContours(mask_uint8.copy(), [box] ,0, (0,0,0), 1)
    
    
    #show_img(mask)
    
    
    mask_uint8 = cv2.erode(mask_uint8, kernel, iterations=1)
    bin_mask = mask_uint8 > 0
    if np.count_nonzero(bin_mask) > 0:
        mask_uint8 = bin2uint(clean_small(bin_mask, max(max_area(bin_mask)/area_diff_thres, 10)))

        #show_img(mask_uint8)
        D = dist_eq(mask_uint8)
        #show_img(D)
        _, localMax = cv2.threshold(D, 0.7*D.max(), 255, 0)
        localMax = cv2.dilate(localMax, kernel, iterations=2)

        #show_img(localMax)
        markers = ndimage.label(localMax, structure=np.ones((3, 3)))[0]
        labels = watershed(-D, markers, mask=bin2uint(base))
        #show_img(labels)
        #labels = fill_con(labels)
        return np.uint8(labels)
    else:
        return np.zeros_like(mask)
    

In [ ]:
SEGM_THRESH = 0.95
SEGM_BASE_THRESH = 0.75
DT_THRESH = 0.001
PROB_THRESH = 0.001

def postprocess(pred, p):
    ret = None
    if p > PROB_THRESH:
        dt = (1 + pred[3,:,:] - pred[4, :, :])/2
        dt = np.clip(dt, 0, 1.0)
        mask = pred[0, :, :] > SEGM_THRESH
        boundaries = pred[1, :, :] > 0.5
        background = pred[2, :, :] > SEGM_THRESH
        #show_img(boundaries)
        #gauss = pred[5, :, :] > 0.1
        mask = np.logical_and(mask, np.logical_not(boundaries))  # Mask-boundaries
        #mask = np.logical_or(mask, gauss)  # Mask+gauss - finds small
        base = pred[0, :, :] > SEGM_BASE_THRESH
        #base = np.logical_and(base, np.logical_not(boundaries))  # Mask-boundaries
        #base = np.logical_or(base, gauss)
        if np.sum(base)>0:
            ret = waters(mask, base, dt, boundaries)
    else:
        pass
    return ret

In [ ]:
# # In case you want to make a mean prediction uncomment following lines:

# save_paths = ['../../data/models/linknet34_folds_008/fold_0/']
# models = [load_model(get_best_model_path(path)) for path in save_paths]

# def predict_mean(inp):
#     preds = [model.nn_module(inp) for model in models]
#     imgs = torch.mean(torch.stack([p[0] for p in preds]), dim=0)
#     probs = torch.mean(torch.stack([p[1] for p in preds]), dim=0)
#     return [imgs, probs]

In [ ]:
F_W = 0.95
F_H = 0.85
def draw_on_image(img, mask):
    for i in np.unique(mask)[1:]:
        _, contours, hierarchy = cv2.findContours(bin2uint(mask==i),
                                                  cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        for cnt in contours:
            rect = cv2.minAreaRect(cnt)
            if rect[1][0]<rect[1][1]:
                hw = (rect[1][0], rect[1][1])
            else:
                hw = (rect[1][0], rect[1][1])
            rect = (rect[0], hw, rect[2])
            box = cv2.boxPoints(rect)
            box = np.int0(box)
            img = cv2.drawContours(img.copy(), [box] ,0, (255,0,0), 1, cv2.LINE_AA)
    return img
        

s = 3
for n, img in enumerate(deploy_loader):
    print(n)
    if n < s:
        n+=1
#         continue
    with torch.no_grad():
        pred = model.nn_module(img.cuda())#predict_mean(img.cuda())#
        print(pred[0].shape)
        print(pred[1].shape)
        for i in range(pred[0].shape[0]):
            img_i = img[i, ...]
            pred_i = pred[0][i, ...].data.cpu()
            pred_i_p = round(pred[1][i].data.cpu().item(), 5)
            print("Prediction", pred_i_p)
            show_img_tensor(img_i)
                #show_trg_tensor(pred_i)
            ret = postprocess(pred_i.numpy(), pred_i_p)
            if ret is not None:
                #show_img(ret)
#                 im = (np.moveaxis(img_i.numpy(), 0, -1)*255).astype(np.uint8)
                im = (np.moveaxis(img_i.numpy(), 0, -1)[:,:,::-1]*255).astype(np.uint8)
                ret_im = np.stack([np.uint8(ret/np.max(ret)*255),
                                   np.zeros((768, 768), dtype=np.uint8),
                                   np.zeros((768, 768), dtype=np.uint8)], axis=2)
                dst = cv2.addWeighted(im, 1.0, ret_im, 0.5, 0)
                show_img(dst)
                show_img(draw_on_image(dst, ret))
            #show_trg_tensor(pred_i)
    break
torch.cuda.empty_cache()

## Make a submit file

In [ ]:
def multi_rle_encode(img):
    labels = label(img)
    return multi_rle_encode_labled(labels)

def multi_rle_encode_labled(labels):
    return [rle_encode(labels==k) for k in np.unique(labels[labels>0])]

In [ ]:
# This should be done in case you made a list of images without ships (visually for example)
stop_list = []
with open("../../data/double_corrected_empty.txt", "r") as f: 
    for line in f: 
        stop_list.append(line[:-1])
print(len(stop_list))

In [ ]:
test_dir = '../../data/test/'
test_names = os.listdir(test_dir)
for img in test_names:
    print(img in stop_list)

In [ ]:
df = pd.DataFrame(columns=['ImageId','EncodedPixels'])
test_names = os.listdir(test_dir)

transform = ImageToTensor()
n = 0
for img in tqdm.tqdm(test_names):
    if img in stop_list:
        df = df.append({'ImageId':img, 'EncodedPixels':np.NaN}, ignore_index=True)
    else:
        img_path = os.path.join(test_dir, img)
        img_tensor = transform(cv2.imread(img_path)).unsqueeze(0)
        with torch.no_grad():
            pred = predict_mean(img_tensor.cuda())

        for i in range(pred[0].shape[0]):
            pred_i = pred[0][i, ...].data.cpu()
            pred_i_p = round(pred[1][i].data.cpu().item(), 5)
            ret = postprocess(pred_i.numpy(), pred_i_p)
            rles = []
            if ret is not None:
                rles = multi_rle_encode_labled(ret)
            if len(rles)>0:
                for rle in rles:
                    df = df.append({'ImageId':img, 'EncodedPixels':rle}, ignore_index=True)
                #img_i = (np.moveaxis(img_tensor[i, ...].numpy(), 0, -1)*255).astype(np.uint8)
                #cv2.imwrite(os.path.join('../../data/vis/subm6/', img), draw_on_image(img_i, ret)[:,:,::-1])
            else:
                df = df.append({'ImageId':img, 'EncodedPixels':np.NaN}, ignore_index=True)

In [ ]:
df.to_csv('../../data/submissions/submission9.csv', header=True, index=False)

In [ ]:
len(df.ImageId.unique())

In [ ]:
len(test_names)